# Speech-to-Text CTC Malay + Singlish + Mandarin

Encoder model + CTC loss for Malay + Singlish + Mandarin languages

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [malaya-speech/example/stt-ctc-model-3mixed](https://github.com/huseinzol05/malaya-speech/tree/master/example/stt-ctc-model-3mixed).
    
</div>

<div class="alert alert-warning">

This module is not language independent, so it not save to use on different languages. Pretrained models trained on hyperlocal languages.
    
</div>

<div class="alert alert-warning">

This is an application of malaya-speech Pipeline, read more about malaya-speech Pipeline at [malaya-speech/example/pipeline](https://github.com/huseinzol05/malaya-speech/tree/master/example/pipeline).
    
</div>

<div class="alert alert-warning">

This interface deprecated, use HuggingFace interface instead.

</div>

In [4]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [1]:
import malaya_speech
import numpy as np
from malaya_speech import Pipeline

`pyaudio` is not available, `malaya_speech.streaming.stream` is not able to use.


In [5]:
import warnings
warnings.filterwarnings('default')

### List available CTC model

In [6]:
malaya_speech.stt.ctc.available_transformer()

/home/husein/dev/malaya-speech/malaya_speech/stt/ctc.py:90: DeprecationWarning: `malaya.stt.ctc.available_transformer` is deprecated, use `malaya.stt.ctc.available_huggingface` instead
  warnings.warn(


,Size (MB),Quantized Size (MB),WER,CER,WER-LM,CER-LM,Language
hubert-conformer-tiny,36.6,10.3,0.495701,0.160176,0.454527,0.139993,[malay]
hubert-conformer,115,31.1,0.334249,0.109406,0.271096,0.096299,[malay]
hubert-conformer-large,392,100,0.280827,0.093818,0.214541,0.081755,[malay]
hubert-conformer-large-3mixed,392,100,0.241126,0.078794,0.132761,0.057482,"[malay, singlish, mandarin]"
best-rq-conformer-tiny,36.6,10.3,0.457089,0.146452,0.421319,0.134883,[malay]
best-rq-conformer,115,31.1,0.342601,0.114661,0.328645,0.119162,[malay]
best-rq-conformer-large,392,100,0.295924,0.101464,0.358809,0.128952,[malay]


### Load CTC transformer model

```python
def transformer(
    model: str = 'hubert-conformer', quantized: bool = False, **kwargs
):
    """
    Load Encoder-CTC ASR model.

    Parameters
    ----------
    model : str, optional (default='hubert-conformer')
        Model architecture supported. Allowed values:

        * ``'hubert-conformer-tiny'`` - Finetuned HuBERT Conformer TINY.
        * ``'hubert-conformer'`` - Finetuned HuBERT Conformer.
        * ``'hubert-conformer-large'`` - Finetuned HuBERT Conformer LARGE.
        * ``'hubert-conformer-large-3mixed'`` - Finetuned HuBERT Conformer LARGE for (Malay + Singlish + Mandarin) languages.
        * ``'best-rq-conformer-tiny'`` - Finetuned BEST-RQ Conformer TINY.
        * ``'best-rq-conformer'`` - Finetuned BEST-RQ Conformer.
        * ``'best-rq-conformer-large'`` - Finetuned BEST-RQ Conformer LARGE.


    quantized : bool, optional (default=False)
        if True, will load 8-bit quantized model.
        Quantized model not necessary faster, totally depends on the machine.

    Returns
    -------
    result : malaya_speech.model.tf.Wav2Vec2_CTC class
    """
```

In [7]:
model = malaya_speech.stt.ctc.transformer(model = 'hubert-conformer-large-3mixed')

/home/husein/dev/malaya-speech/malaya_speech/stt/ctc.py:126: DeprecationWarning: `malaya.stt.ctc.transformer` is deprecated, use `malaya.stt.ctc.huggingface` instead
  warnings.warn(
2023-01-16 23:20:54.674559: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 23:20:54.679759: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-16 23:20:54.679781: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2023-01-16 23:20:54.679785: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2023-01-16 23:20:54.679881: I tensorflow/stream_ex

### Load sample

In [8]:
ceramah, sr = malaya_speech.load('speech/khutbah/wadi-annuar.wav')
record1, sr = malaya_speech.load('speech/record/savewav_2020-11-26_22-36-06_294832.wav')
record2, sr = malaya_speech.load('speech/record/savewav_2020-11-26_22-40-56_929661.wav')
singlish0, sr = malaya_speech.load('speech/singlish/singlish0.wav')
singlish1, sr = malaya_speech.load('speech/singlish/singlish1.wav')
singlish2, sr = malaya_speech.load('speech/singlish/singlish2.wav')
mandarin0, sr = malaya_speech.load('speech/mandarin/597.wav')
mandarin1, sr = malaya_speech.load('speech/mandarin/584.wav')
mandarin2, sr = malaya_speech.load('speech/mandarin/509.wav')

In [9]:
import IPython.display as ipd

ipd.Audio(ceramah, rate = sr)

As we can hear, the speaker speaks in kedahan dialects plus some arabic words, let see how good our model is.

In [10]:
ipd.Audio(record1, rate = sr)

In [11]:
ipd.Audio(record2, rate = sr)

In [12]:
ipd.Audio(singlish0, rate = sr)

In [13]:
ipd.Audio(singlish1, rate = sr)

In [14]:
ipd.Audio(singlish2, rate = sr)

In [15]:
ipd.Audio(mandarin0, rate = sr)

In [16]:
ipd.Audio(mandarin1, rate = sr)

In [17]:
ipd.Audio(mandarin2, rate = sr)

### Predict using greedy decoder

```python
def greedy_decoder(self, inputs):
    """
    Transcribe inputs using greedy decoder.

    Parameters
    ----------
    input: List[np.array]
        List[np.array] or List[malaya_speech.model.frame.Frame].

    Returns
    -------
    result: List[str]
    """
```

In [18]:
%%time

model.greedy_decoder([ceramah, record1, record2, singlish0, singlish1, singlish2, 
                      mandarin0, mandarin1, mandarin2])

2023-01-16 23:21:22.637128: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 589805568 exceeds 10% of free system memory.
2023-01-16 23:21:22.790337: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 589805568 exceeds 10% of free system memory.
2023-01-16 23:21:22.933908: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 589805568 exceeds 10% of free system memory.
2023-01-16 23:21:22.981193: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 589805568 exceeds 10% of free system memory.
2023-01-16 23:21:23.307972: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 589805568 exceeds 10% of free system memory.


Instructions for updating:
Use `tf.cast` instead.
CPU times: user 1min 4s, sys: 15 s, total: 1min 19s
Wall time: 13.3 s


['jadi dalam perjalanan ini lunia yang susah ini ketika nabi mengajar muaz bin jabar tadi ni alah maini',
 'helo nama saya sin saya taksukamandi saya masang',
 'helo nama saya husin saya suka mandi say mond t hari',
 'and then se how they ro it in film okay actualy',
 'then you tat to your eyes',
 'severson involve',
 'gei wo lai ge zhang jie zui xin de ge',
 'wo xiang shou kan jiang su yi shi pin dao de jie mu',
 'qiu yi shou ge de ming zi ge ci li you zhuan sheng yi meng wang shi qing shi xiu']

### Predict using beam decoder

```python
def beam_decoder(self, inputs, beam_width: int = 100):
    """
    Transcribe inputs using beam decoder.

    Parameters
    ----------
    input: List[np.array]
        List[np.array] or List[malaya_speech.model.frame.Frame].
    beam_width: int, optional (default=100)
        beam size for beam decoder.

    Returns
    -------
    result: List[str]
    """
```

In [19]:
%%time

model.beam_decoder([ceramah, record1, record2, singlish0, singlish1, singlish2, 
                      mandarin0, mandarin1, mandarin2])

CPU times: user 1min 20s, sys: 25.7 s, total: 1min 45s
Wall time: 38.9 s


['jadi dalam perjalanan ini lunia yang susah ini ketika nabi mengajar muaz bin jabar tadi ni alah maini',
 'helo nama saya sine saya taksukamandi k saya masang',
 'helo nama saya husin saya suka mandi saya mond hari',
 'and then se how they roe it in film okay actualy',
 ' then you tat to your eys',
 'severson involve',
 'gei wo lai ge zhang jie zui xin de ge',
 'wo xiang shou kan jiang su yin shi pin dao de jie mu',
 'qiu yi shou ge de ming zi ge ci li you zhuan sheng yi meng wang shi qing shi xiu']

### Predict logits

```python
def predict_logits(self, inputs):
    """
    Predict logits from inputs.

    Parameters
    ----------
    input: List[np.array]
        List[np.array] or List[malaya_speech.model.frame.Frame].


    Returns
    -------
    result: List[np.array]
    """
```

In [21]:
%%time

logits = model.predict_logits([ceramah, record1, record2, singlish0, singlish1, singlish2, 
                      mandarin0, mandarin1, mandarin2])

CPU times: user 1min 17s, sys: 25.7 s, total: 1min 43s
Wall time: 38.9 s


In [22]:
logits, logits[0].shape, logits[1].shape

([array([[5.0769908e-13, 4.9920995e-06, 4.1646995e-06, ..., 4.5173483e-11,
          4.9810876e-07, 9.9995333e-01],
         [7.0174210e-13, 1.1965726e-06, 8.6241844e-06, ..., 4.4535094e-11,
          3.0639339e-07, 9.9991900e-01],
         [2.3351788e-13, 5.8303417e-06, 1.9618931e-06, ..., 1.1045688e-11,
          7.3586186e-08, 9.9996752e-01],
         ...,
         [2.5784982e-10, 1.1928772e-03, 1.1550728e-05, ..., 8.8894708e-08,
          8.0232326e-07, 9.9622393e-01],
         [2.8769603e-11, 2.7891362e-04, 5.6442818e-06, ..., 3.6080590e-09,
          3.2085375e-06, 9.9853903e-01],
         [4.6427840e-12, 4.9945329e-06, 1.7572627e-05, ..., 1.3756351e-09,
          8.1958979e-06, 9.9879241e-01]], dtype=float32),
  array([[7.3830272e-15, 2.6149910e-06, 2.9792586e-08, ..., 2.6439647e-13,
          1.7397490e-05, 9.9995226e-01],
         [4.1778101e-14, 2.1268688e-06, 1.0971124e-08, ..., 1.6074101e-12,
          3.9539827e-04, 9.9956423e-01],
         [2.3152864e-10, 1.3606093e-06, 2

### Load Language Model

```python
def language_model(
    model: str = 'dump-combined', **kwargs
):
    """
    Load KenLM language model.

    Parameters
    ----------
    model : str, optional (default='dump-combined')
        Model architecture supported. Allowed values:

        * ``'bahasa'`` - Gathered from malaya-speech ASR bahasa transcript.
        * ``'bahasa-news'`` - Gathered from malaya-speech ASR bahasa transcript + Bahasa News (Random sample 300k sentences).
        * ``'bahasa-combined'`` - Gathered from malaya-speech ASR bahasa transcript + Bahasa News (Random sample 300k sentences) + Bahasa Wikipedia (Random sample 150k sentences).
        * ``'redape-community'`` - Mirror for https://github.com/redapesolutions/suara-kami-community
        * ``'dump-combined'`` - Academia + News + IIUM + Parliament + Watpadd + Wikipedia + Common Crawl + training set from https://github.com/huseinzol05/malay-dataset/tree/master/dumping/clean.
        * ``'manglish'`` - Manglish News + Manglish Reddit + Manglish forum + training set from https://github.com/huseinzol05/malay-dataset/tree/master/dumping/clean.
        * ``'bahasa-manglish-combined'`` - Combined `dump-combined` and `manglish`.

    Returns
    -------
    result : str
    """
```

For 3 mixed languages, I prefer to load `bahasa-manglish-combined`.

In [19]:
from pyctcdecode import build_ctcdecoder
from ctc_decoders import Scorer
from ctc_decoders import ctc_beam_search_decoder
from malaya_speech.utils.char import CTC_VOCAB
import kenlm

In [ ]:
lm = malaya_speech.language_model.kenlm(model = 'bahasa-manglish-combined')

Downloading:   0%|          | 0.00/608M [00:00<?, ?B/s]

In [30]:
kenlm_model = kenlm.Model(lm)
decoder = build_ctcdecoder(
    CTC_VOCAB + ['_'],
    kenlm_model,
    alpha=0.2,
    beta=1.0,
    ctc_token_idx=len(CTC_VOCAB)
)

In [31]:
for no, l in enumerate(logits):
    out = decoder.decode_beams(l, prune_history=True)
    d_lm, lm_state, timesteps, logit_score, lm_score = out[0]
    print(no, d_lm)

0 jadi dalam perjalanan ini dunia yang susah ini ketika nabi mengajar muadz bin jabal tadi ni allah ma ini
1 hello nama saya san saya tak suka mandi saya masang
2 hello nama saya hussin saya suka mandi saya mandi hari
3 and then see how they ro it in film okay actually
4 then you tat to your eyes
5 severson involve
6 gei wo lai ge zhang jie zui xin de ge
7 nansenrnrnibaunwoixiagshoukanjiangsugyigshipindaodejiemu
8 qiu yi shou ge de ming zi ge ci li you zhuan sheng yi wen wang shi qing shi xiu


In [34]:
scorer = Scorer(0.5, 1.0, lm, CTC_VOCAB)

In [35]:
for no, l in enumerate(logits):
    o = ctc_beam_search_decoder(l, CTC_VOCAB, 20, ext_scoring_func = scorer)[0][1]
    print(no, o)

0 jadi dalam perjalanan ini dunia yang susah ini ketika nabi mengajar muadz bin jabal tadi ni allah ma ini
1 hello nama saya san saya tak sumanding
2 hello nama saya hussin saya suka mandi saman hari
3 and then see how they row it in film okay actually
4 then you tat to your eyes
5 severson involve
6 gei wo lai ge zhang jie zui xin de ge
7 wo xiang shou kan jiang su yi shi pin dao de jie mu
8 qiu yi shou ge de ming zi ge ci li you zhuan sheng yi wen wang shi qing shi xiu
